In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder

In [2]:
ls

CONTRIBUTING.md    README.md          data/              student.ipynb
LICENSE.md         Round 1.ipynb      halfway-there.gif


In [3]:
data = pd.read_csv("data/kc_house_data.csv")

In [4]:
data = data.reset_index()

In [5]:
columns = data.columns.to_list()
columns

['index',
 'id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [6]:
data.columns = data.columns.str.replace(' ', '')

In [7]:
data = data.drop(columns=['index', 'id'])

In [8]:
data.shape

(21597, 20)

In [9]:
data.dtypes

date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront       float64
view             float64
condition          int64
grade              int64
sqft_above         int64
sqft_basement     object
yr_built           int64
yr_renovated     float64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [10]:
data['date'] = pd.to_datetime(data['date'])

In [11]:
# reorder so price is the first column
cols = list(data.columns)
cols = [cols[1]] + cols[:1] + cols[2:]
data = data[cols]

In [18]:
data.head()

,price,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
1,538000.0,2014-12-09,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
3,604000.0,2014-12-09,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,510000.0,2015-02-18,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
5,1230000.0,2014-05-12,4,4.50,5420,101930,1.0,0.0,0.0,3,11,3890,1530.0,2001,0.0,98053,47.6561,-122.005,4760,101930
6,257500.0,2014-06-27,3,2.25,1715,6819,2.0,0.0,0.0,3,7,1715,?,1995,0.0,98003,47.3097,-122.327,2238,6819


In [20]:
data.to_csv('cleaned_kc_house_data.csv')

### Need to check on possibly modification of data types for the following columns: 

1.) sqft_basement

2.) date              

In [13]:
data.isna().mean().round(4) * 100

price             0.00
date              0.00
bedrooms          0.00
bathrooms         0.00
sqft_living       0.00
sqft_lot          0.00
floors            0.00
waterfront       11.00
view              0.29
condition         0.00
grade             0.00
sqft_above        0.00
sqft_basement     0.00
yr_built          0.00
yr_renovated     17.79
zipcode           0.00
lat               0.00
long              0.00
sqft_living15     0.00
sqft_lot15        0.00
dtype: float64

### Check why waterfront and year renovated have a missing data percentage over 10%

In [14]:
errors = []
for idx in data.index:
    try: 
        float(data.sqft_basement[idx])
    except:
        errors.append(idx)

In [15]:
data.iloc[errors].sqft_basement.value_counts()

?    454
Name: sqft_basement, dtype: int64

In [16]:
data = data.dropna()

In [17]:
data.isna().mean().round(4) * 100

price            0.0
date             0.0
bedrooms         0.0
bathrooms        0.0
sqft_living      0.0
sqft_lot         0.0
floors           0.0
waterfront       0.0
view             0.0
condition        0.0
grade            0.0
sqft_above       0.0
sqft_basement    0.0
yr_built         0.0
yr_renovated     0.0
zipcode          0.0
lat              0.0
long             0.0
sqft_living15    0.0
sqft_lot15       0.0
dtype: float64

### Finding Correlation

In [ ]:
data.describe()

In [ ]:
corr = data.corr()
corr

In [ ]:
sns.heatmap(corr);

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig1, ax1 = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, ax=ax1, cmap="viridis");

In [ ]:
positively_correlated_cols = ['price','bedrooms','bathrooms', 'sqft_living', 'sqft_lot']
positively_correlated_df = data[positively_correlated_cols]
sns.pairplot(positively_correlated_df);

In [ ]:
log_bedrooms = np.log(positively_correlated_df['bedrooms'])
log_bathrooms = np.log(positively_correlated_df['bathrooms'])
log_sqft_living = np.log(positively_correlated_df['sqft_living'])
log_sqft_lot = np.log(positively_correlated_df['sqft_lot'])

scaled_bedrooms = (log_bedrooms-np.mean(log_bedrooms))/np.sqrt(np.var(log_bedrooms))
scaled_bathrooms = (log_bathrooms-np.mean(log_bathrooms))/np.sqrt(np.var(log_bathrooms))
scaled_sqft_living = (log_sqft_living-np.mean(log_sqft_living))/np.sqrt(np.var(log_sqft_living))
scaled_sqft_lot = (log_sqft_lot-np.mean(log_sqft_lot))/np.sqrt(np.var(log_sqft_lot))


data_positively_correlated_transformed = pd.DataFrame([])
data_positively_correlated_transformed['bedrooms'] = scaled_bedrooms
data_positively_correlated_transformed['bathrooms'] = scaled_bathrooms
data_positively_correlated_transformed['sqft_living'] = scaled_sqft_living
data_positively_correlated_transformed['sqft_lot'] = scaled_sqft_lot
data_positively_correlated_transformed['price'] = positively_correlated_df['price']

display(data_positively_correlated_transformed.shape)
display(data_positively_correlated_transformed.head())
sns.pairplot(data_positively_correlated_transformed);

In [ ]:
model_1_df = data_positively_correlated_transformed[['price','bedrooms','bathrooms', 'sqft_living', 'sqft_lot']].copy()
model_1_df.dropna(inplace=True)

In [ ]:
model_1 = ols(formula = "price ~ bedrooms + bathrooms + sqft_living + sqft_lot", data = model_1_df)
model_1_results = model_1.fit()

In [ ]:
model_1_results.summary()

### Model 1 ~ data_positively_correlated_transformed

We are only explaining about 39% of the variance in price, but we have 4 positively transformed features and it's statistically significant at an alpha of 0.05.

According to our model:

A home price with zero bedrooms, bathrooms, square feet living and square lot on average is expected to cost $541,300.

*** A negative coefficient suggests that as the independent variable increases, the dependent variable tends to decrease.

### Linearity - data_positively_correlated_transformed

In [ ]:
rainbow_statistic, rainbow_p_value = linear_rainbow(model_1_results)
print("Rainbow statistic:", rainbow_statistic)
print("Rainbow p-value:", rainbow_p_value)

The null hypothesis is that the model is linearly predicted by the features, alternative hypothesis is that it is not. Thus returning a p-value above the alpha of .05 means that the current model meets the linearity assumption. 

### Normality - data_positively_correlated_transformed

Linear regression assumes that the residuals are normally distributed. It is possible to check this qualitatively with a Q-Q plot, but this example shows how to assess it statistically.

The Jarque-Bera test is performed automatically as part of the model summary output, labeled Jarque-Bera (JB) and Prob(JB).

The null hypothesis is that the residuals are normally distributed, alternative hypothesis is that they are not. Thus returning a low p-value means that the current model violates the normality assumption.

From Model 1 Results: The Prob(JB): 0.00

Thus returning a p-value below the alpha of .05 means that the current model does not meet the normality assumption.


In [ ]:
from statsmodels.graphics.regressionplots import plot_leverage_resid2
fig, ax = plt.subplots(figsize=(8,6))
fig = plot_leverage_resid2(model_1_results, ax = ax)

### Homoscadasticity - data_positively_correlated_transformed

Linear regression assumes that the variance of the dependent variable is homogeneous across different value of the independent variable(s). We can visualize this by looking at the predicted life expectancy vs. the residuals.

In [ ]:
y = data_positively_correlated_transformed['price']
y_hat = model_1_results.predict()

fig2, ax2 = plt.subplots()
ax2.set(xlabel="Predicted Home Price",
        ylabel="Residuals (Predicted - Home Price)")
ax2.scatter(x=y_hat, y=y_hat-y, color="blue", alpha=0.2);

Just visually inspecting this, it seems like our model over-predicts home price between .75e6 and 1.00e6 in a way that doesn't happen for other homes. 

Plus we have some weird-looking data in the upper end that we might want to inspect.

In [ ]:
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(y_hat, data_positively_correlated_transformed['price'])
print("Lagrange Multiplier p-value:", lm_p_value)
print("F-statistic p-value:", f_p_value)